In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('/content/drive/MyDrive/treino/classificação de rotatividade de clientes/Customer Churn Classification.csv')
df.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure_Months,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Credit_Score_Category,Tenure_Group,Loyalty_Score,Exited
0,1,15634602,Hargrave,619,France,Female,42,24,0.00,1,Yes,Yes,101348.88,Good,1-3 years,2,1
1,2,15647311,Hill,608,Spain,Female,41,12,83807.86,1,No,Yes,112542.58,Good,0-1 year,1,0


In [3]:
def build_neural_network(input_shape, layers_config, activation='relu', dropout_rate=0.0, output_activation='sigmoid', learning_rate=0.001):
    """
    Função para criar uma rede neural com base em parâmetros fornecidos.

    Parâmetros:
    - input_shape (int): O número de features de entrada (dimensão dos dados).
    - layers_config (list): Uma lista com o número de neurônios em cada camada oculta.
    - activation (str): A função de ativação para as camadas ocultas (default: 'relu').
    - dropout_rate (float): A taxa de dropout aplicada após cada camada (default: 0.0).
    - output_activation (str): A função de ativação para a camada de saída (default: 'sigmoid' para classificação binária).
    - learning_rate (float): A taxa de aprendizado para o otimizador (default: 0.001).

    Retorna:
    - model: Um modelo Keras pronto para compilar e treinar.
    """

    model = Sequential()

    # Adicionar a primeira camada oculta com a entrada especificada
    model.add(Dense(layers_config[0], activation=activation, input_shape=(input_shape,)))

    # Adicionar dropout, se especificado
    if dropout_rate > 0:
        model.add(Dropout(dropout_rate))

    # Adicionar outras camadas ocultas conforme especificado na lista layers_config
    for units in layers_config[1:]:
        model.add(Dense(units, activation=activation))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))

    # Camada de saída (para classificação binária)
    model.add(Dense(1, activation=output_activation))

    # Compilar o modelo
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [4]:
def fit_neural(model, X_train, y_train, X_test, y_test, epochs=20, batch_size=32):
  history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))
  y_pred = (model.predict(X_test) > 0.5).astype('int32')
  print(classification_report(y_test, y_pred))

In [5]:
X = df.drop(['RowNumber', 'CustomerId', 'Surname', 'Exited'], axis=1)
y = df['Exited']

In [6]:
categorical_features = ['Geography', 'Gender', 'HasCrCard', 'IsActiveMember', 'Credit_Score_Category', 'Tenure_Group']
numerical_features = ['CreditScore', 'Age', 'Tenure_Months', 'Balance', 'NumOfProducts', 'EstimatedSalary', 'Loyalty_Score']

In [7]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

Rede Neural com duas camadas uma com 64 neuronios e outra 32 neuronios treinado 20 epocas batch_size=32

In [10]:
input_shape = X_train.shape[1]
layers_config = [64, 32]

In [11]:
model = build_neural_network(input_shape, layers_config)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
fit_neural(model, X_train, y_train, X_test, y_test)

Epoch 1/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7297 - loss: 0.5215 - val_accuracy: 0.8365 - val_loss: 0.3788
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8506 - loss: 0.3708 - val_accuracy: 0.8515 - val_loss: 0.3627
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8429 - loss: 0.3685 - val_accuracy: 0.8590 - val_loss: 0.3484
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8655 - loss: 0.3313 - val_accuracy: 0.8570 - val_loss: 0.3496
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8537 - loss: 0.3458 - val_accuracy: 0.8585 - val_loss: 0.3439
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8620 - loss: 0.3291 - val_accuracy: 0.8555 - val_loss: 0.3474
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8559 - loss: 0.3312 - val_accuracy: 0.8560 - val_loss: 0.3524
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8614 - loss: 0.3249 - val_accuracy: 0.

In [13]:
fit_neural(model, X_train, y_train, X_test, y_test, epochs=50)

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8789 - loss: 0.3009 - val_accuracy: 0.8590 - val_loss: 0.3472
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8762 - loss: 0.2997 - val_accuracy: 0.8565 - val_loss: 0.3459
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8742 - loss: 0.3019 - val_accuracy: 0.8545 - val_loss: 0.3507
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8714 - loss: 0.3012 - val_accuracy: 0.8575 - val_loss: 0.3447
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8779 - loss: 0.2986 - val_accuracy: 0.8530 - val_loss: 0.3510
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8806 - loss: 0.2914 - val_accuracy: 0.8590 - val_loss: 0.3460
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8758 - loss: 0.2875 - val_accuracy: 0.8505 - val_loss: 0.3513
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8749 - loss: 0.2927 - val_accuracy: 0.

In [14]:
fit_neural(model, X_train, y_train, X_test, y_test, epochs=100)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8975 - loss: 0.2416 - val_accuracy: 0.8440 - val_loss: 0.4025
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8988 - loss: 0.2367 - val_accuracy: 0.8320 - val_loss: 0.4105
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8971 - loss: 0.2322 - val_accuracy: 0.8390 - val_loss: 0.4034
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9009 - loss: 0.2372 - val_accuracy: 0.8435 - val_loss: 0.3958
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9045 - loss: 0.2295 - val_accuracy: 0.8410 - val_loss: 0.3975
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9027 - loss: 0.2346 - val_accuracy: 0.8515 - val_loss: 0.4113
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8940 - loss: 0.2396 - val_accuracy: 0.8425 - val_loss: 0.4040
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8992 - loss: 0.2346 - val_accu

In [15]:
fit_neural(model, X_train, y_train, X_test, y_test, epochs=200)

Epoch 1/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9407 - loss: 0.1502 - val_accuracy: 0.8265 - val_loss: 0.5677
Epoch 2/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9356 - loss: 0.1511 - val_accuracy: 0.8165 - val_loss: 0.5775
Epoch 3/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9366 - loss: 0.1550 - val_accuracy: 0.8315 - val_loss: 0.5642
Epoch 4/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9400 - loss: 0.1478 - val_accuracy: 0.8225 - val_loss: 0.5713
Epoch 5/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9381 - loss: 0.1544 - val_accuracy: 0.8400 - val_loss: 0.5748
Epoch 6/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9379 - loss: 0.1531 - val_accuracy: 0.8340 - val_loss: 0.5587
Epoch 7/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9412 - loss: 0.1446 - val_accuracy: 0.8140 - val_loss: 0.5773
Epoch 8/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9331 - loss: 0.1597 - val_accu

In [16]:
fit_neural(model, X_train, y_train, X_test, y_test, batch_size=64)

Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9800 - loss: 0.0593 - val_accuracy: 0.8180 - val_loss: 1.0429
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9863 - loss: 0.0509 - val_accuracy: 0.8160 - val_loss: 1.0561
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9870 - loss: 0.0500 - val_accuracy: 0.8235 - val_loss: 1.0514
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9865 - loss: 0.0504 - val_accuracy: 0.8190 - val_loss: 1.0573
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9842 - loss: 0.0544 - val_accuracy: 0.8220 - val_loss: 1.0456
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9848 - loss: 0.0547 - val_accuracy: 0.8190 - val_loss: 1.0602
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9853 - loss: 0.0538 - val_accuracy: 0.8165 - val_loss: 1.0655
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9837 - loss: 0.0557 - val_accuracy: 0.

In [17]:
fit_neural(model, X_train, y_train, X_test, y_test, epochs=50, batch_size=64)

Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9830 - loss: 0.0502 - val_accuracy: 0.8115 - val_loss: 1.1099
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9789 - loss: 0.0601 - val_accuracy: 0.8155 - val_loss: 1.1044
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9828 - loss: 0.0559 - val_accuracy: 0.8075 - val_loss: 1.1191
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9828 - loss: 0.0573 - val_accuracy: 0.8110 - val_loss: 1.1096
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9880 - loss: 0.0485 - val_accuracy: 0.8195 - val_loss: 1.1109
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9829 - loss: 0.0541 - val_accuracy: 0.8130 - val_loss: 1.1111
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9833 - loss: 0.0567 - val_accuracy: 0.8255 - val_loss: 1.1093
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9824 - loss: 0.0536 - val_accuracy: 0.

In [18]:
fit_neural(model, X_train, y_train, X_test, y_test, epochs=100, batch_size=64)

Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9833 - loss: 0.0475 - val_accuracy: 0.8080 - val_loss: 1.2602
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9832 - loss: 0.0515 - val_accuracy: 0.8035 - val_loss: 1.2434
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9879 - loss: 0.0438 - val_accuracy: 0.8145 - val_loss: 1.2229
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9882 - loss: 0.0406 - val_accuracy: 0.8155 - val_loss: 1.2426
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9877 - loss: 0.0447 - val_accuracy: 0.8145 - val_loss: 1.2330
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9856 - loss: 0.0457 - val_accuracy: 0.8160 - val_loss: 1.2458
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9880 - loss: 0.0453 - val_accuracy: 0.8105 - val_loss: 1.2512
Epoch 8/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9881 - loss: 0.0453 - val_accu

In [19]:
model = build_neural_network(input_shape, layers_config, activation='tanh')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
fit_neural(model, X_train, y_train, X_test, y_test)

Epoch 1/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7558 - loss: 0.4992 - val_accuracy: 0.8195 - val_loss: 0.3956
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8295 - loss: 0.4046 - val_accuracy: 0.8415 - val_loss: 0.3732
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8372 - loss: 0.3811 - val_accuracy: 0.8475 - val_loss: 0.3673
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8492 - loss: 0.3673 - val_accuracy: 0.8535 - val_loss: 0.3505
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8580 - loss: 0.3490 - val_accuracy: 0.8615 - val_loss: 0.3465
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8617 - loss: 0.3403 - val_accuracy: 0.8620 - val_loss: 0.3406
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8565 - loss: 0.3394 - val_accuracy: 0.8620 - val_loss: 0.3399
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8623 - loss: 0.3289 - val_accuracy: 0.

In [21]:
fit_neural(model, X_train, y_train, X_test, y_test, epochs=50)

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8677 - loss: 0.3117 - val_accuracy: 0.8650 - val_loss: 0.3338
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8674 - loss: 0.3123 - val_accuracy: 0.8650 - val_loss: 0.3371
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8659 - loss: 0.3130 - val_accuracy: 0.8650 - val_loss: 0.3328
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8665 - loss: 0.3090 - val_accuracy: 0.8650 - val_loss: 0.3341
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8675 - loss: 0.3172 - val_accuracy: 0.8670 - val_loss: 0.3340
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8668 - loss: 0.3070 - val_accuracy: 0.8620 - val_loss: 0.3361
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8706 - loss: 0.3042 - val_accuracy: 0.8670 - val_loss: 0.3333
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8728 - loss: 0.2993 - val_accuracy: 0.

In [22]:
fit_neural(model, X_train, y_train, X_test, y_test, epochs=100)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9095 - loss: 0.2276 - val_accuracy: 0.8565 - val_loss: 0.3677
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8999 - loss: 0.2376 - val_accuracy: 0.8580 - val_loss: 0.3719
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9065 - loss: 0.2283 - val_accuracy: 0.8580 - val_loss: 0.3656
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9070 - loss: 0.2333 - val_accuracy: 0.8580 - val_loss: 0.3677
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9058 - loss: 0.2286 - val_accuracy: 0.8575 - val_loss: 0.3715
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9038 - loss: 0.2363 - val_accuracy: 0.8565 - val_loss: 0.3716
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9093 - loss: 0.2265 - val_accuracy: 0.8515 - val_loss: 0.3761
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8996 - loss: 0.2372 - val_accu

In [23]:
fit_neural(model, X_train, y_train, X_test, y_test, batch_size=64)

Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9708 - loss: 0.1000 - val_accuracy: 0.8385 - val_loss: 0.5709
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9675 - loss: 0.0994 - val_accuracy: 0.8345 - val_loss: 0.5740
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9706 - loss: 0.0944 - val_accuracy: 0.8360 - val_loss: 0.5726
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9729 - loss: 0.0914 - val_accuracy: 0.8355 - val_loss: 0.5738
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9701 - loss: 0.1021 - val_accuracy: 0.8370 - val_loss: 0.5788
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9742 - loss: 0.0888 - val_accuracy: 0.8345 - val_loss: 0.5786
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9700 - loss: 0.0983 - val_accuracy: 0.8345 - val_loss: 0.5836
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9711 - loss: 0.0938 - val_accuracy: 0.

In [24]:
fit_neural(model, X_train, y_train, X_test, y_test, batch_size=128)

Epoch 1/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9770 - loss: 0.0825 - val_accuracy: 0.8350 - val_loss: 0.6052
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9773 - loss: 0.0798 - val_accuracy: 0.8335 - val_loss: 0.6065
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9775 - loss: 0.0804 - val_accuracy: 0.8320 - val_loss: 0.6083
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9755 - loss: 0.0832 - val_accuracy: 0.8345 - val_loss: 0.6091
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9765 - loss: 0.0812 - val_accuracy: 0.8335 - val_loss: 0.6125
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9773 - loss: 0.0840 - val_accuracy: 0.8300 - val_loss: 0.6152
Epoch 7/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9775 - loss: 0.0822 - val_accuracy: 0.8340 - val_loss: 0.6127
Epoch 8/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9788 - loss: 0.0799 - val_accuracy: 0.8335 - val_loss:

In [35]:
model = build_neural_network(input_shape, layers_config, dropout_rate=0.2)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
fit_neural(model, X_train, y_train, X_test, y_test, epochs=50)

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7958 - loss: 0.4841 - val_accuracy: 0.8285 - val_loss: 0.3906
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8244 - loss: 0.4082 - val_accuracy: 0.8525 - val_loss: 0.3587
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8398 - loss: 0.3923 - val_accuracy: 0.8540 - val_loss: 0.3481
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8461 - loss: 0.3689 - val_accuracy: 0.8565 - val_loss: 0.3466
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8524 - loss: 0.3641 - val_accuracy: 0.8575 - val_loss: 0.3408
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8591 - loss: 0.3401 - val_accuracy: 0.8570 - val_loss: 0.3464
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8591 - loss: 0.3527 - val_accuracy: 0.8550 - val_loss: 0.3449
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8608 - loss: 0.3509 - val_accuracy: 0.

In [39]:
model = build_neural_network(input_shape, layers_config, dropout_rate=0.2, learning_rate=0.0001)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [46]:
fit_neural(model, X_train, y_train, X_test, y_test, epochs=20, batch_size=64)

Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8662 - loss: 0.3236 - val_accuracy: 0.8645 - val_loss: 0.3383
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8692 - loss: 0.3206 - val_accuracy: 0.8645 - val_loss: 0.3383
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8636 - loss: 0.3324 - val_accuracy: 0.8640 - val_loss: 0.3381
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8684 - loss: 0.3302 - val_accuracy: 0.8650 - val_loss: 0.3383
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8650 - loss: 0.3294 - val_accuracy: 0.8640 - val_loss: 0.3387
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8663 - loss: 0.3264 - val_accuracy: 0.8640 - val_loss: 0.3386
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8666 - loss: 0.3300 - val_accuracy: 0.8635 - val_loss: 0.3383
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8642 - loss: 0.3331 - val_accuracy: 0.

In [48]:
model.save('modelo_classificacao.keras')